In [21]:
%cd /raid/ruslan_bazhenov/cargoxray/

/raid/ruslan_bazhenov/cargoxray


In [22]:
import shutil
from pathlib import Path

import albumentations as alb
import cv2
import numpy as np
import pandas as pd
import torchvision
from PIL import Image
from torchvision import transforms as tf
import torch
import tqdm


In [23]:
images = pd.read_json('data_new/images.json.gz',
                        orient='records',
                        typ='frame',
                        compression='gzip')
images = images.set_index('image_id')
images

,filepath,md5,size,width,height,old_filename,old_location
image_id,,,,,,,
0,data/images/00000.tif,3ab70521b3fdcd223949fda05b60f2f0,15229794,4399,1154,327010D01201909060002.tif,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
1,data/images/00001.tif,4873919d851f8bcc8421e3dd6a8c7781,14862814,4293,1154,327010D01201909100004.tif,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
2,data/images/00002.tif,b1ddc51d2a1cf08e7d619bb9e4bda828,13287580,3838,1154,327010D01201909100037.tif,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
3,data/images/00003.tif,187a39ed319c8506d959906f3b510abe,13291042,3839,1154,327010D01201909120012.tif,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
4,data/images/00004.tif,26594213171935bb519219fc95ff6b89,13332586,3851,1154,327010D01201909030024.tif,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
...,...,...,...,...,...,...,...
15193,data/images/03B59.tif,1b469be64a9c6963a6f714a0a89225c9,8390562,3496,1200,image2704 (576).tif,raw_backup/old/Готовые для передачи 2806/для п...
15194,data/images/03B5A.tif,6332a0f7a1f8effba22169a2217955a8,10257762,4274,1200,image2704 (540).tif,raw_backup/old/Готовые для передачи 2806/для п...
15195,data/images/03B5B.tif,fccb2a0320903b505703c6221af92781,8047362,3353,1200,image2704 (563).tif,raw_backup/old/Готовые для передачи 2806/для п...


In [24]:
annotations = pd.read_json('data_new/annotations.json.gz',
                            orient='records',
                            typ='frame',
                            compression='gzip')
annotations = annotations.set_index('bbox_id')
annotations

,image_id,x_min,y_min,width,height,label
bbox_id,,,,,,
0,470,2760,115,416,893,scooter
1,470,846,637,324,177,scooter
2,455,2077,877,1261,210,spare parts
3,455,672,126,154,67,motorbike
4,114,2116,691,1161,337,tools
...,...,...,...,...,...,...
41998,14366,127,73,4064,750,shoes
41999,14387,122,129,2912,630,clothes
42000,14342,131,119,1156,729,clothes


In [25]:
annotations = annotations\
    .reset_index()\
    .merge(images['old_location'],
           on='image_id')\
    .drop_duplicates(['bbox_id',
                      'image_id'])\
    .set_index('bbox_id')\
    .rename(columns={'old_location': 'filepath'})
annotations


,image_id,x_min,y_min,width,height,label,filepath
bbox_id,,,,,,,
0,470,2760,115,416,893,scooter,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
1,470,846,637,324,177,scooter,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
2,455,2077,877,1261,210,spare parts,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
3,455,672,126,154,67,motorbike,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
4,114,2116,691,1161,337,tools,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
...,...,...,...,...,...,...,...
41998,14366,127,73,4064,750,shoes,raw_backup/old/clean/2606/image2704 (573).tif
41999,14387,122,129,2912,630,clothes,raw_backup/old/clean/2606/image2704 (575).tif
42000,14342,131,119,1156,729,clothes,raw_backup/old/clean/2606/image2704 (577).tif


In [26]:
annotations = annotations.replace((
        'brokkoli',
        'clouthes',
        'equpment',
        'Equipment',
        'grapes',
        'motobike',
        'motorcycle',
        'nectarine',
        'pears',
        'scooters',
        'textiles',
        'tomates',
        'Household goods',
        'Lamps',
        'Car wheels',
        'Clothes',
        'Shoes',
        'Spare parts',
        'appliances',
        'car wheels ',
        'carweels',
        'carwheels',
        'cars',
        'equipment '
    ), (
        'broccoli',
        'clothes',
        'equipment',
        'equipment',
        'grape',
        'bike',
        'bike',
        'nectarin',
        'pear',
        'scooter',
        'textile',
        'tomato',
        'household goods',
        'lamps',
        'car wheels',
        'clothes',
        'shoes',
        'spare parts',
        'appliance',
        'car wheels',
        'car wheels',
        'car wheels',
        'car',
        'equipment'
    ))

In [27]:
labels = annotations[['label', 'filepath']]\
    .drop_duplicates()\
    .groupby('label')\
    .count()['filepath']\
    .sort_values(ascending=False)


labels.name = 'labels'
ignored_labels = labels.iloc[20:]
ignored_labels

label
lamps                36
garlic               35
home appliances      33
bags                 31
pineapple            29
                     ..
agriculturaltires     1
leek                  1
lettuce               1
motorbike parts       1
finished products     1
Name: labels, Length: 81, dtype: int64

In [28]:
ignored_annotations = annotations\
    .reset_index()\
    .merge(ignored_labels,
           on='label')\
    .drop(columns=['labels'])\
    .set_index('bbox_id')

ignored_annotations

,image_id,x_min,y_min,width,height,label,filepath
bbox_id,,,,,,,
3,455,672,126,154,67,motorbike,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
723,61,2802,306,399,308,motorbike,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
724,61,3315,316,372,249,motorbike,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
1158,459,3513,762,319,292,motorbike,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
1207,640,1388,808,360,197,motorbike,raw_backup/new/Нурик1092/Ainur2308/Nurik2308/3...
...,...,...,...,...,...,...,...
32860,12516,1328,107,676,865,scateboard,raw_backup/old/clean/3103/MIxed222.tif
32861,12516,3346,131,548,737,scateboard,raw_backup/old/clean/3103/MIxed222.tif
32853,12516,546,103,151,100,ball,raw_backup/old/clean/3103/MIxed222.tif


In [29]:
ignored_images = ignored_annotations['filepath'].drop_duplicates()
ignored_images

bbox_id
3        raw_backup/new/Нурик1092/Ainur2308/327010D0120...
723      raw_backup/new/Нурик1092/Ainur2308/327010D0120...
1158     raw_backup/new/Нурик1092/Ainur2308/327010D0120...
1207     raw_backup/new/Нурик1092/Ainur2308/Nurik2308/3...
360      raw_backup/new/Нурик1092/Ainur2308/327010D0120...
                               ...                        
32467             raw_backup/old/clean/05/mixed12 (66).tif
32797             raw_backup/old/clean/3103/Mixed (13).tif
32806             raw_backup/old/clean/3103/Mixed (16).tif
33425              raw_backup/old/clean/3003/mixed (3).tif
32852               raw_backup/old/clean/3103/MIxed222.tif
Name: filepath, Length: 474, dtype: object

In [30]:
ignored_annotations = annotations\
    .reset_index()\
    .merge(ignored_images,
           on='filepath')\
    .set_index('bbox_id')
ignored_annotations


,image_id,x_min,y_min,width,height,label,filepath
bbox_id,,,,,,,
2,455,2077,877,1261,210,spare parts,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
3,455,672,126,154,67,motorbike,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
358,126,423,120,1377,918,tableware,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
359,126,2466,144,1102,964,spare parts,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
360,126,1977,133,492,374,bags,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
...,...,...,...,...,...,...,...
41647,8334,812,320,190,115,equipment,raw_backup/old/clean/1706/image2704 (302).tif
41648,8334,2456,233,508,392,shoes,raw_backup/old/clean/1706/image2704 (302).tif
41649,8334,2024,456,1187,550,clothes,raw_backup/old/clean/1706/image2704 (302).tif


In [31]:
selected_annotations = pd.concat([annotations, ignored_annotations])\
    .drop_duplicates(keep=False)

selected_annotations

,image_id,x_min,y_min,width,height,label,filepath
bbox_id,,,,,,,
0,470,2760,115,416,893,scooter,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
1,470,846,637,324,177,scooter,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
4,114,2116,691,1161,337,tools,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
5,114,322,133,174,300,tools,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
6,114,1100,166,249,358,tools,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
...,...,...,...,...,...,...,...
41998,14366,127,73,4064,750,shoes,raw_backup/old/clean/2606/image2704 (573).tif
41999,14387,122,129,2912,630,clothes,raw_backup/old/clean/2606/image2704 (575).tif
42000,14342,131,119,1156,729,clothes,raw_backup/old/clean/2606/image2704 (577).tif


In [32]:
selected_images = selected_annotations['filepath']\
    .drop_duplicates()\
    .reset_index(drop=True)
selected_images


0       raw_backup/new/Нурик1092/Ainur2308/327010D0120...
1       raw_backup/new/Нурик1092/Ainur2308/327010D0120...
2       raw_backup/new/Нурик1092/Ainur2308/327010D0120...
3       raw_backup/new/Нурик1092/Ainur2308/Nurik2308/3...
4       raw_backup/new/Нурик1092/Ainur2308/327010D0120...
                              ...                        
9459        raw_backup/old/clean/2606/image2704 (571).tif
9460        raw_backup/old/clean/2606/image2704 (572).tif
9461        raw_backup/old/clean/2606/image2704 (573).tif
9462        raw_backup/old/clean/2606/image2704 (575).tif
9463        raw_backup/old/clean/2606/image2704 (577).tif
Name: filepath, Length: 9464, dtype: object

In [33]:
selected_labels = selected_annotations['label']\
    .drop_duplicates()\
    .reset_index(drop=True)
    
selected_labels.index.name = 'label_id'

', '.join(f"'{lbl}'" for lbl in selected_labels.to_list())

"'scooter', 'tools', 'spare parts', 'toys', 'shoes', 'textile', 'fabrics', 'tableware', 'car wheels', 'auto parts', 'tires', 'equipment', 'bicycle', 'appliance', 'clothes', 'grape', 'pepper', 'nectarin', 'tomato', 'household goods'"

In [34]:
train_images = selected_images.sample(frac=0.85)
train_images

600     raw_backup/new/Нурик1092/Ainur2308/Nurik2308/3...
2659    raw_backup/new/готовые для передачи 01072021/с...
9437        raw_backup/old/clean/2606/image2704 (542).tif
8516        raw_backup/old/clean/250126-391341/350115.tif
2272    raw_backup/new/готовые для передачи 01072021/с...
                              ...                        
1789    raw_backup/new/2308/2308/327010D01202001110008...
8170                      raw_backup/old/clean/081643.tif
452     raw_backup/new/Нурик1092/Ainur2308/327010D0120...
4555          raw_backup/old/clean/13/Pictures  (761).tif
8300                      raw_backup/old/clean/142032.tif
Name: filepath, Length: 8044, dtype: object

In [35]:
val_images = pd.concat([selected_images, train_images])\
    .drop_duplicates(keep=False)
val_images

5       raw_backup/new/Нурик1092/Ainur2308/327010D0120...
10      raw_backup/new/Нурик1092/Ainur2308/327010D0120...
12      raw_backup/new/Нурик1092/Ainur2308/327010D0120...
15      raw_backup/new/Нурик1092/Ainur2308/327010D0120...
24      raw_backup/new/Нурик1092/Ainur2308/Nurik2308/3...
                              ...                        
9425        raw_backup/old/clean/2606/image2704 (480).tif
9427        raw_backup/old/clean/2606/image2704 (548).tif
9428        raw_backup/old/clean/2606/image2704 (547).tif
9436        raw_backup/old/clean/2606/image2704 (492).tif
9439        raw_backup/old/clean/2606/image2704 (495).tif
Name: filepath, Length: 1420, dtype: object

In [36]:
train_images.to_json('yolo/train_images.json', orient='records')
val_images.to_json('yolo/val_images.json', orient='records')

In [16]:
train_annotations = selected_annotations\
    .reset_index()\
    .merge(train_images, on='filepath')\
    .set_index('bbox_id')
train_annotations


,image_id,x_min,y_min,width,height,label,filepath
bbox_id,,,,,,,
4,114,2116,691,1161,337,tools,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
5,114,322,133,174,300,tools,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
6,114,1100,166,249,358,tools,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
7,114,843,264,85,258,tools,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
8,384,506,510,1271,513,spare parts,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
...,...,...,...,...,...,...,...
41998,14366,127,73,4064,750,shoes,raw_backup/old/clean/2606/image2704 (573).tif
41999,14387,122,129,2912,630,clothes,raw_backup/old/clean/2606/image2704 (575).tif
42000,14342,131,119,1156,729,clothes,raw_backup/old/clean/2606/image2704 (577).tif


In [17]:
val_annotations = selected_annotations\
    .reset_index()\
    .merge(val_images, on='filepath')\
    .set_index('bbox_id')
val_annotations


,image_id,x_min,y_min,width,height,label,filepath
bbox_id,,,,,,,
0,470,2760,115,416,893,scooter,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
1,470,846,637,324,177,scooter,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
24,231,609,624,1086,411,tools,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
25,231,2030,630,1274,456,spare parts,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
28,240,540,143,1324,866,shoes,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
...,...,...,...,...,...,...,...
41987,14362,3583,304,1669,660,textile,raw_backup/old/clean/2606/image2704 (562).tif
41988,14362,2430,152,1183,831,textile,raw_backup/old/clean/2606/image2704 (562).tif
41994,14349,588,492,1096,383,spare parts,raw_backup/old/clean/2606/image2704 (571).tif


In [18]:
print(len(train_annotations['label'].drop_duplicates()))
print(len(val_annotations['label'].drop_duplicates()))

20
20


In [19]:
weights = 1 / train_annotations[['label', 'filepath']]\
    .drop_duplicates()\
    .groupby('label')\
    .count()\
    .rename(columns={'filepath': 'weight'})['weight']\
    .sort_values()

weights


label
tomato             0.041667
nectarin           0.033333
bicycle            0.014085
household goods    0.011111
car wheels         0.009709
scooter            0.009259
grape              0.008333
pepper             0.006993
appliance          0.004762
tools              0.004464
tires              0.003704
auto parts         0.003401
tableware          0.003115
equipment          0.001754
toys               0.001122
fabrics            0.000948
spare parts        0.000932
clothes            0.000888
textile            0.000788
shoes              0.000594
Name: weight, dtype: float64

In [20]:
weights = train_annotations[['label', 'filepath']]\
    .drop_duplicates()\
    .merge(weights,on='label')\
    .groupby('filepath')\
    .mean()['weight']
weights


filepath
raw_backup/new/0409completed/327010D01201911060024.tif                                            0.004132
raw_backup/new/0409completed/327010D01201911060025.tif                                            0.001343
raw_backup/new/0409completed/327010D01201911060026.tif                                            0.000932
raw_backup/new/0409completed/327010D01201911060027.tif                                            0.001343
raw_backup/new/0409completed/327010D01201911060028.tif                                            0.001754
                                                                                                    ...   
raw_backup/old/Интеллект21/Интеллект21/Женя/Новая папка/Снимки ИДК для проекта ИИ/mixed 67.tif    0.000594
raw_backup/old/Интеллект21/Интеллект21/Женя/Новая папка/Снимки ИДК для проекта ИИ/mixed 70.tif    0.000594
raw_backup/old/Интеллект21/Интеллект21/Женя/Новая папка/Снимки ИДК для проекта ИИ/mixed 73.tif    0.000594
raw_backup/old/Интеллект21/И

In [21]:

balanced_train_images = weights.sample(n=100000, replace=True, weights=weights).index.to_series()
balanced_train_images

filepath
raw_backup/old/clean/05/shoes (251).tif                                                                            raw_backup/old/clean/05/shoes (251).tif
raw_backup/old/clean/1-67_json951+1404/mixed 875.tif                                                     raw_backup/old/clean/1-67_json951+1404/mixed 8...
raw_backup/new/Нурик1092/Нурик2708/327010D01201912250012.tif                                             raw_backup/new/Нурик1092/Нурик2708/327010D0120...
raw_backup/old/clean/240321_json/mixed with textile (297).tif                                            raw_backup/old/clean/240321_json/mixed with te...
raw_backup/old/clean/240321_json/mixed with textile (321).tif                                            raw_backup/old/clean/240321_json/mixed with te...
                                                                                                                               ...                        
raw_backup/old/clean/13/Mixed819 (1).tif                     

In [22]:
bbox_params = alb.BboxParams(format='yolo',
                             label_fields=('labels',))

aug = alb.Compose(
    transforms=[

        alb.SmallestMaxSize(max_size=600),

        alb.Affine(scale=1,
                   rotate=3,
                   shear=3,
                   mode=cv2.BORDER_REFLECT101,
                   p=0.25),

        alb.SmallestMaxSize(max_size=500),

        alb.RandomCrop(500, 500),

        alb.HorizontalFlip(p=0.25),

        alb.Equalize(p=0.25),

        alb.RandomGamma(gamma_limit=(85, 115),
                        p=0.25),

    ],
    bbox_params=bbox_params
)

In [23]:
# # Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
# path: ../datasets/coco128  # dataset root dir
# train: images/train2017  # train images (relative to 'path') 128 images
# val: images/train2017  # val images (relative to 'path') 128 images
# test:  # test images (optional)

# # Classes
# nc: 80  # number of classes
# names: [ 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
#          'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
#          'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
#          'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
#          'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
#          'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
#          'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
#          'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
#          'hair drier', 'toothbrush' ]  # class names


Path('yolo/dataset.yaml').write_text(
    'train: train/images\n'
    'val: val/images\n'
    '\n'
    'nc: 20\n'
    'names: [ ' + ', '.join(f"'{lbl}'" for lbl in selected_labels.to_list()) + ' ]\n'
    
)

286

In [ ]:
for idx, img_loc in tqdm.tqdm(enumerate(balanced_train_images),
                              total=len(balanced_train_images)):

    image = Image.open(img_loc)
    if image.mode == 'I;16':
        image = image.convert('I').point(
            [i/256 for i in range(2**16)], 'L')
    else:
        image = image.convert('L')

    sel = train_annotations.loc[train_annotations['filepath'] == img_loc].copy(
    )

    sel['x'] = sel['x_min'] + sel['width'] // 2
    sel['y'] = sel['y_min'] + sel['height'] // 2

    sel['x'] /= image.width
    sel['y'] /= image.height
    sel['width'] /= image.width
    sel['height'] /= image.height

    bboxes = sel[['x', 'y', 'width', 'height']
                 ].to_records(index=False).tolist()
    labels = [selected_labels.index[selected_labels == lbl][0]
              for lbl in sel['label'].tolist()]

    sample = {
        'image': np.asarray(image),
        'bboxes': bboxes,
        'labels': labels,
    }

    image.close()
    try:

        transformed_sample = aug(**sample)
    except:
        print(img_loc)
        continue

    image = Image.fromarray(transformed_sample['image'])
    image.save(f'yolo/train/images/{idx:05X}.png')
    image.close()

    bboxes = transformed_sample['bboxes']
    labels = transformed_sample['labels']

    bboxes = '\n'.join([
        f'{lbl} {box[0]} {box[1]} {box[2]} {box[3]}' for box, lbl in zip(bboxes, labels)]
    )

    Path(f'yolo/train/labels/{idx:05X}.txt').write_text(bboxes)


    # bboxes = [(int((x-w/2)*500),
    #            int((y-h/2)*500),
    #            int((x+w/2)*500),
    #            int((y+h/2)*500),) for x, y, w, h, in transformed_sample['bboxes']]

    # wbboxes = tf.ToPILImage()(torchvision.utils.draw_bounding_boxes(
    #     torch.from_numpy(transformed_sample['image']).expand(3, -1, -1),
    #     boxes=torch.IntTensor(bboxes),
    #     labels=[str(l) for l in transformed_sample['labels']],
    #     fill=True,
    #     colors=['red'] * len(bboxes),
    # ))
    
    # display(wbboxes)


  0%|                                                                                                                                                                 | 56/100000 [00:08<4:15:25,  6.52it/s]

raw_backup/old/clean/1-551t_json0118/Mixed522 (1).tif


  0%|▎                                                                                                                                                               | 221/100000 [00:33<4:07:46,  6.71it/s]

raw_backup/new/готовые для передачи 01072021/снимки2921/Готовые снимки(634шт)/Готовые4/Mixed (16).tif


  0%|▊                                                                                                                                                               | 479/100000 [01:10<3:43:20,  7.43it/s]